# Random recommendation (whole)

In [3]:
import pandas as pd
import random
pd.options.mode.chained_assignment = None # hide SettingWithCopyWarning

df = pd.read_csv('D:/data_encoding_sel.csv')

In [5]:
def rec_ran10(train,test,epoch,recall_num):
    train_items = list(set(df['mod_itemfullname']))
    test_actual = list(test['mod_itemfullname'])
    T, M = 0, 0

    for epo in range(epoch):
        s = 0
        MRR_res = 0
        for i in range(len(test)):
            rand = random.sample(train_items,len(train_items)) # random recommended list
            if test_actual[i] in rand:
                idx = rand.index(test_actual[i])
                MRR_res += 1/(idx+1)
                if test_actual[i] in rand[0:recall_num]:
                    s+=1
        rec10 = s/len(test)
        MRR_res = MRR_res/len(test)
        T += rec10
        M += MRR_res
    T = T/epoch
    M = M/epoch
    print(f'Average recall@{recall_num} in {epoch} epochs:\n',T)
    print(f'Average MRR in {epoch} epochs:',M)
    
# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
rec_ran10(train,test,epoch=200,recall_num=3)

Average recall@3 in 200 epochs:
 0.004094292803970231
Average MRR in 200 epochs: 0.00979217395957716


# Random recommendation (week)

In [7]:
df['user_id'] = pd.to_datetime(df['transaction_date']).apply(lambda time: time.dayofweek)

In [8]:
def rec_ranweek(train,test,epoch,recall_num):
    test_actual = list(test['mod_itemfullname'])
    test_userid = list(test['user_id'])
    Each_day = {}
    for i in range(7):
        Each_day[i] = list(set(train[train['user_id']==list(train['user_id'])[i]]['mod_itemfullname'])) # i=0 : monday

    T, M = 0, 0

    for epo in range(epoch):    
        s=0
        MRR_res = 0
        for i in range(len(test_actual)):
            idx = test_userid[i]
            rand = random.sample(Each_day[idx],len(Each_day[idx]))
            if test_actual[i] in rand:
                p = rand.index(test_actual[i])
                MRR_res += 1/(p+1)
                if test_actual[i] in rand[0:recall_num]:
                    s+=1    
        rec10 = s/len(test_actual)
        MRR_res = MRR_res/len(test)
        T += rec10
        M += MRR_res
    T = T/epoch
    M = M/epoch

    print(f'Average recall@{recall_num} in {epoch} epochs:',rec10)
    print(f'Average MRR in {epoch} epochs:',M)
    
# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
rec_ranweek(train,test,200,3)

Average recall@3 in 200 epochs: 0.007444168734491315
Average MRR in 200 epochs: 0.013724528735855598


# Random recommendation (Time)

In [9]:
from datetime import datetime
df['user_id']= [0 for i in range(len(df))]

for i in range(len(df)):
    d_target = datetime.strptime(df['transaction_date'][i][11:], "%H:%M:%S")

    if datetime.strptime('08:00:00', "%H:%M:%S") <= d_target < datetime.strptime('11:00:00', "%H:%M:%S"):
        df['user_id'][i] = 0
    elif datetime.strptime('11:00:00', "%H:%M:%S") <= d_target < datetime.strptime('14:00:00', "%H:%M:%S"):
        df['user_id'][i] = 1 
    elif datetime.strptime('14:00:00', "%H:%M:%S") <= d_target < datetime.strptime('17:00:00', "%H:%M:%S"):
        df['user_id'][i] = 2 
    elif datetime.strptime('17:00:00', "%H:%M:%S") <= d_target < datetime.strptime('20:00:00', "%H:%M:%S"):
        df['user_id'][i] = 3 
    elif datetime.strptime('20:00:00', "%H:%M:%S") <= d_target < datetime.strptime('23:59:59', "%H:%M:%S"):
        df['user_id'][i] = 4
    elif datetime.strptime('00:00:00', "%H:%M:%S") <= d_target < datetime.strptime('08:00:00', "%H:%M:%S"):
        df['user_id'][i] = 5

In [10]:
# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
rec_ranweek(train,test,200,3)

Average recall@3 in 200 epochs: 0.015715467328370553
Average MRR in 200 epochs: 0.019066633215279235


In [11]:
# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
rec_ranweek(train,test,200,5)

Average recall@5 in 200 epochs: 0.02315963606286187
Average MRR in 200 epochs: 0.018863313339500917


# Random recommendation (week+time)

In [12]:

df['user_id1'] = pd.to_datetime(df['transaction_date']).apply(lambda time: time.dayofweek)
df['user_id'] = [0 for i in range(len(df))]
for i in range(len(df['transaction_date'])):
    d_target = datetime.strptime(df['transaction_date'][i][11:], "%H:%M:%S")
    
    for j in range(7):
        if df['user_id1'][i]==j:  # Monday
            if datetime.strptime('08:00:00', "%H:%M:%S") <= d_target < datetime.strptime('11:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6
            elif datetime.strptime('11:00:00', "%H:%M:%S") <= d_target < datetime.strptime('14:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+1 
            elif datetime.strptime('14:00:00', "%H:%M:%S") <= d_target < datetime.strptime('17:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+2  
            elif datetime.strptime('17:00:00', "%H:%M:%S") <= d_target < datetime.strptime('20:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+3  
            elif datetime.strptime('20:00:00', "%H:%M:%S") <= d_target < datetime.strptime('23:59:59', "%H:%M:%S"):
                df['user_id'][i] = j*6+4 
            elif datetime.strptime('00:00:00', "%H:%M:%S") <= d_target < datetime.strptime('08:00:00', "%H:%M:%S"):
                df['user_id'][i] = j*6+5

In [13]:
def rec_ranweek(train,test,epoch,recall_num):
    test_actual = list(test['mod_itemfullname'])
    test_userid = list(test['user_id'])
    Each_day = {}
    for i in range(42):
        Each_day[i] = list(set(train[train['user_id']==list(train['user_id'])[i]]['mod_itemfullname'])) 

    T, M = 0, 0

    for epo in range(epoch):    
        s=0
        MRR_res = 0
        for i in range(len(test_actual)):
            idx = test_userid[i]
            rand = random.sample(Each_day[idx],len(Each_day[idx]))
            if test_actual[i] in rand:
                p = rand.index(test_actual[i])
                MRR_res += 1/(p+1)
                if test_actual[i] in rand[0:recall_num]:
                    s+=1    
        rec10 = s/len(test_actual)
        MRR_res = MRR_res/len(test)
        T += rec10
        M += MRR_res
    T = T/epoch
    M = M/epoch
    print(f'Average recall@{recall_num} in {epoch} epochs:',rec10)
    print(f'Average MRR in {epoch} epochs:',M)

# 80 train, 20 test
train = df[0:4836]
test = df[4836:]
rec_ranweek(train,test,200,3)

Average recall@3 in 200 epochs: 0.009925558312655087
Average MRR in 200 epochs: 0.01786431559390088
